In [1]:
import numpy as np
import pandas as pd
import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd
import math
# visualization
import cv2
from glob import glob
# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix, roc_curve
import timm
# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torch.optim.swa_utils import AveragedModel, SWALR
from transformers import get_cosine_schedule_with_warmup
from collections import defaultdict
# import matplotlib.pyplot as plt
# Albumentations for augmentations
import albumentations as A
import albumentations
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    seed = 1
    model_name = "tf_efficientnetv2_b2"
    train_bs = 16
    valid_bs = 64
    image_size = 1024
    epochs = 25
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(CFG.device)

cuda:0


In [3]:
df = pd.read_csv("train_5folds.csv")
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,fold
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,1
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,1
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,1
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,1
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,0


In [4]:
is_hol = df['cancer'] == 1
df_try = df[is_hol]
df1 = df.append([df_try]*1,ignore_index=True)
print(len(df1))

55864


In [5]:
def init_logger(log_file='train1.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()
now = datetime.now()
datetime_now = now.strftime("%m/%d/%Y, %H:%M:%S")
LOGGER.info(f"Date :{datetime_now}")

Date :01/23/2023, 08:29:04


In [6]:
from albumentations import DualTransform
image_size = 1024
def isotropically_resize_image(img, size, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC):
    h, w = img.shape[:2]
    if max(w, h) == size:
        return img
    if w > h:
        scale = size / w
        h = h * scale
        w = size
    else:
        scale = size / h
        w = w * scale
        h = size
    interpolation = interpolation_up if scale > 1 else interpolation_down
    resized = cv2.resize(img, (int(w), int(h)), interpolation=interpolation)
    return resized


class IsotropicResize(DualTransform):
    def __init__(self, max_side, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC,
                 always_apply=False, p=1):
        super(IsotropicResize, self).__init__(always_apply, p)
        self.max_side = max_side
        self.interpolation_down = interpolation_down
        self.interpolation_up = interpolation_up

    def apply(self, img, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC, **params):
        return isotropically_resize_image(img, size=self.max_side, interpolation_down=interpolation_down,
                                          interpolation_up=interpolation_up)

    def apply_to_mask(self, img, **params):
        return self.apply(img, interpolation_down=cv2.INTER_NEAREST, interpolation_up=cv2.INTER_NEAREST, **params)

    def get_transform_init_args_names(self):
        return ("max_side", "interpolation_down", "interpolation_up")
    
data_transforms = {
    "train": A.Compose([
        # A.Resize(image_size, image_size),
        # IsotropicResize(max_side = image_size),
        # A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
        # A.RandomBrightnessContrast(),
        # A.VerticalFlip(p=0.5),   
        # A.ColorJitter(),
        # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        # A.HorizontalFlip(p=0.5),
        # A.Cutout(max_h_size=int(image_size * 0.1), max_w_size=int(image_size * 0.1), num_holes=5, p=0.5),
        A.VerticalFlip(p=0.5),   
        A.ColorJitter(),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Cutout(max_h_size=int(image_size * 0.1), max_w_size=int(image_size * 0.1), num_holes=5, p=0.5),
        # A.OneOf([ 
        # A.OpticalDistortion(distort_limit=1.0), 
        # A.GridDistortion(num_steps=5, distort_limit=1.),
        # A.ElasticTransform(alpha=3), ], p=0.2),
        # A.OneOf([
            # A.GaussNoise(var_limit=[10, 50]),
            # A.GaussianBlur(),
            # A.MotionBlur(),
            # A.MedianBlur(), ], p=0.2),
        # A.OneOf([
        #     A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
        #     A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
        #     A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        # ], p=0.25),
        # A.CoarseDropout(max_holes=8, max_height=image_size//20, max_width=image_size//20,
        #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        # A.Normalize(mean=0, std=1),
        ToTensorV2(),], p=1.0),
    
    "valid": A.Compose([
        # IsotropicResize(max_side =image_size),
        # A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
        # A.Normalize(mean=0, std=1),
        # A.Resize(image_size, image_size),
        ToTensorV2(),
        ], p=1.0)
}

LOGGER.info(f"train transform{data_transforms['train']}")


train transformCompose([
  VerticalFlip(always_apply=False, p=0.5),
  ColorJitter(always_apply=False, p=0.5, brightness=[0.8, 1.2], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=[-0.2, 0.2]),
  ShiftScaleRotate(always_apply=False, p=0.5, shift_limit_x=(-0.0625, 0.0625), shift_limit_y=(-0.0625, 0.0625), scale_limit=(-0.050000000000000044, 0.050000000000000044), rotate_limit=(-10, 10), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box'),
  HorizontalFlip(always_apply=False, p=0.5),
  Cutout(always_apply=False, p=0.5, num_holes=5, max_h_size=102, max_w_size=102),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=False),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


In [7]:
# from albumentations import DualTransform
# image_size = 1024
# def isotropically_resize_image(img, size, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC):
#     h, w = img.shape[:2]
#     if max(w, h) == size:
#         return img
#     if w > h:
#         scale = size / w
#         h = h * scale
#         w = size
#     else:
#         scale = size / h
#         w = w * scale
#         h = size
#     interpolation = interpolation_up if scale > 1 else interpolation_down
#     resized = cv2.resize(img, (int(w), int(h)), interpolation=interpolation)
#     return resized


# class IsotropicResize(DualTransform):
#     def __init__(self, max_side, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC,
#                  always_apply=False, p=1):
#         super(IsotropicResize, self).__init__(always_apply, p)
#         self.max_side = max_side
#         self.interpolation_down = interpolation_down
#         self.interpolation_up = interpolation_up

#     def apply(self, img, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC, **params):
#         return isotropically_resize_image(img, size=self.max_side, interpolation_down=interpolation_down,
#                                           interpolation_up=interpolation_up)

#     def apply_to_mask(self, img, **params):
#         return self.apply(img, interpolation_down=cv2.INTER_NEAREST, interpolation_up=cv2.INTER_NEAREST, **params)

#     def get_transform_init_args_names(self):
#         return ("max_side", "interpolation_down", "interpolation_up")
    
# data_transforms = {
#     "train": A.Compose([
# #         A.Resize(image_size, image_size),
#         # IsotropicResize(max_side = image_size),
#        A.PadIfNeeded(min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
#         albumentations.HorizontalFlip(p=0.5),
#         albumentations.VerticalFlip(p=0.5),
#         # albumentations.RandomBrightness(limit=0.2, p=0.75),
#         # albumentations.RandomContrast(limit=0.2, p=0.75),

#         # albumentations.OneOf([
#         #     albumentations.OpticalDistortion(distort_limit=1.),
#         #     albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         # ], p=0.75),

#         # albumentations.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=40, val_shift_limit=0, p=0.75),
#         albumentations.ShiftScaleRotate(p = 0.5),
#         A.Cutout(always_apply=False, p=0.5, num_holes=5, max_h_size=image_size//10, max_w_size=image_size//10),
#         # A.RandomBrightnessContrast(),
#         # A.VerticalFlip(p=0.5),   
#         A.ColorJitter(p = 0.7),
#         # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
#         # A.HorizontalFlip(p=0.5),
#         # A.Cutout(max_h_size=int(image_size * 0.1), max_w_size=int(image_size * 0.1), num_holes=5, p=0.5),
#         # albumentations.RandomBrightness(limit=0.2, p=0.75),
#         # albumentations.RandomContrast(limit=0.2, p=0.75),

#         # albumentations.OneOf([
#         #     albumentations.OpticalDistortion(distort_limit=1.),
#         #     albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         # ], p=0.75),

#         # albumentations.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=40, val_shift_limit=0, p=0.75),
#         # albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=30, border_mode=0, p=0.75),
#         # A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
#         # A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
#         # A.CLAHE(p=0.5),
#         # albumentations.OneOf([
#         # albumentations.OpticalDistortion(distort_limit=1.),
#         # albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         # ], p=0.75),
#         # A.OneOf([
#         # A.GaussianBlur(),
#         # A.MotionBlur(),
#         # A.MedianBlur(), ], p=0.5),
#         # A.IAASharpen(p = 0.2),
#         # A.JpegCompression(p=0.2),
#         # A.Downscale(scale_min=0.5, scale_max=0.75),
#         # A.OneOf([ A.JpegCompression(), A.Downscale(scale_min=0.1, scale_max=0.15), ], p=0.2), 
#         # A.IAAPiecewiseAffine(),
# #         A.OneOf([ 
# #         A.OpticalDistortion(distort_limit=1.0), 
# #         A.GridDistortion(num_steps=5, distort_limit=1.),
# #         A.ElasticTransform(alpha=3), ], p=0.2),
# #         A.OneOf([
# #             A.GaussNoise(var_limit=[10, 50]),
# #             A.GaussianBlur(),
# #             A.MotionBlur(),
# #             A.MedianBlur(), ], p=0.2),
#         # A.OneOf([
#         #     A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
#         #     A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
#         #     A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
#         # ], p=0.25),
#         # A.CoarseDropout(max_holes=8, max_height=image_size//20, max_width=image_size//20,
#         #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
#         # A.Normalize(mean=0, std=1),
#         ToTensorV2(),], p=1.0),
    
#     "valid": A.Compose([
#         # IsotropicResize(max_side = image_size),
#         A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
#         # A.Normalize(mean=0, std=1),
# #         A.Resize(image_size, image_size),
#         ToTensorV2(),
#         ], p=1.0)
# }

# LOGGER.info(f"train transform{data_transforms['train']}")


In [8]:
def pad(array, target_shape):
    return np.pad(
        array,
        [(0, target_shape[i] - array.shape[i]) for i in range(len(array.shape))],
        "constant",
    )
    
def load_img(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = pad(image, (1024, 800, 3))
        # img = img.reshape((*resize))
    return image
#     image = cv2.resize(image, (320, 320), cv2.INTER_NEAREST)
#     image = image.astype(np.float32)
#     mx = np.max(image)
#     if mx:
#         image/=mx
#     image = image /255.0
    
    return image
class BreastDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = f"flip/{row.patient_id}_{row.image_id}.png"
        img = load_img(img_path)
        label = row['cancer']
        # img = np.transpose(img, (2, 0, 1))
        data = self.transforms(image=img)
        img  = data['image']
        # img = img/255.0
        return torch.tensor(img).float(), torch.tensor(label).long()
        
    def __len__(self):
        return len(self.df)
    
fold0 = df[df['fold']==0]
train_dataset = BreastDataset(fold0, transforms = data_transforms['train'])
image, label = train_dataset[0]
print(image.shape, label)
print(image.max())

torch.Size([3, 1344, 840]) tensor(0)
tensor(255.)


In [9]:

# from pylab import rcParams

# f, axarr = plt.subplots(1,15, figsize = (20, 20))
# imgs = []
# for p in range(15):
#     img, label = train_dataset[p]
#     img = img.transpose(0, 1).transpose(1,2).cpu().numpy()
#     img = img.astype(np.uint8)
#     imgs.append(img)
#     axarr[p].imshow(img)


In [10]:
class Model(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(model_name, pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer, required
import math

class AdamP(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, delta=0.1, wd_ratio=0.1, nesterov=False):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        delta=delta, wd_ratio=wd_ratio, nesterov=nesterov)
        super(AdamP, self).__init__(params, defaults)

    def _channel_view(self, x):
        return x.view(x.size(0), -1)

    def _layer_view(self, x):
        return x.view(1, -1)

    def _cosine_similarity(self, x, y, eps, view_func):
        x = view_func(x)
        y = view_func(y)

        return F.cosine_similarity(x, y, dim=1, eps=eps).abs_()

    def _projection(self, p, grad, perturb, delta, wd_ratio, eps):
        wd = 1
        expand_size = [-1] + [1] * (len(p.shape) - 1)
        for view_func in [self._channel_view, self._layer_view]:

            cosine_sim = self._cosine_similarity(grad, p.data, eps, view_func)

            if cosine_sim.max() < delta / math.sqrt(view_func(p.data).size(1)):
                p_n = p.data / view_func(p.data).norm(dim=1).view(expand_size).add_(eps)
                perturb -= p_n * view_func(p_n * perturb).sum(dim=1).view(expand_size)
                wd = wd_ratio

                return perturb, wd

        return perturb, wd

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad.data
                beta1, beta2 = group['betas']
                nesterov = group['nesterov']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data)
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                # Adam
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']

                state['step'] += 1
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                step_size = group['lr'] / bias_correction1

                if nesterov:
                    perturb = (beta1 * exp_avg + (1 - beta1) * grad) / denom
                else:
                    perturb = exp_avg / denom

                # Projection
                wd_ratio = 1
                if len(p.shape) > 1:
                    perturb, wd_ratio = self._projection(p, grad, perturb, group['delta'], group['wd_ratio'], group['eps'])

                # Weight decay
                if group['weight_decay'] > 0:
                    p.data.mul_(1 - group['lr'] * group['weight_decay'] * wd_ratio)

                # Step
                p.data.add_(perturb, alpha=-step_size)

        return loss

class SGDP(Optimizer):
    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False, eps=1e-8, delta=0.1, wd_ratio=0.1):
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening, weight_decay=weight_decay,
                        nesterov=nesterov, eps=eps, delta=delta, wd_ratio=wd_ratio)
        super(SGDP, self).__init__(params, defaults)

    def _channel_view(self, x):
        return x.view(x.size(0), -1)

    def _layer_view(self, x):
        return x.view(1, -1)

    def _cosine_similarity(self, x, y, eps, view_func):
        x = view_func(x)
        y = view_func(y)

        return F.cosine_similarity(x, y, dim=1, eps=eps).abs_()

    def _projection(self, p, grad, perturb, delta, wd_ratio, eps):
        wd = 1
        expand_size = [-1] + [1] * (len(p.shape) - 1)
        for view_func in [self._channel_view, self._layer_view]:

            cosine_sim = self._cosine_similarity(grad, p.data, eps, view_func)

            if cosine_sim.max() < delta / math.sqrt(view_func(p.data).size(1)):
                p_n = p.data / view_func(p.data).norm(dim=1).view(expand_size).add_(eps)
                perturb -= p_n * view_func(p_n * perturb).sum(dim=1).view(expand_size)
                wd = wd_ratio

                return perturb, wd

        return perturb, wd

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['momentum'] = torch.zeros_like(p.data)

                # SGD
                buf = state['momentum']
                buf.mul_(momentum).add_(grad, alpha=1 - dampening)
                if nesterov:
                    d_p = grad + momentum * buf
                else:
                    d_p = buf

                # Projection
                wd_ratio = 1
                if len(p.shape) > 1:
                    d_p, wd_ratio = self._projection(p, grad, d_p, group['delta'], group['wd_ratio'], group['eps'])

                # Weight decay
                if group['weight_decay'] > 0:
                    p.data.mul_(1 - group['lr'] * group['weight_decay'] * wd_ratio / (1-momentum))

                # Step
                p.data.add_(d_p, alpha=-group['lr'])

        return loss

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [13]:
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau


class GradualWarmupScheduler(_LRScheduler):
    """ Gradually warm-up(increasing) learning rate in optimizer.
    Proposed in 'Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour'.
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        multiplier: target learning rate = base lr * multiplier if multiplier > 1.0. if multiplier = 1.0, lr starts from 0 and ends up with the base_lr.
        total_epoch: target learning rate is reached at total_epoch, gradually
        after_scheduler: after target_epoch, use this scheduler(eg. ReduceLROnPlateau)
    """

    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier < 1.:
            raise ValueError('multiplier should be greater thant or equal to 1.')
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super(GradualWarmupScheduler, self).__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_last_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
        if self.last_epoch <= self.total_epoch:
            warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                param_group['lr'] = lr
        else:
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.total_epoch)

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.total_epoch)
                self._last_lr = self.after_scheduler.get_last_lr()
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

class Lookahead(optim.Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

In [14]:

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    torch.cuda.empty_cache()
    gc.collect()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    truth = []
    pred = []
    global_step = 0
    scaler = GradScaler()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train')
    for step, (images, labels) in pbar:
        optimizer.zero_grad()
        data_time.update(time.time() - end)
        images = images.to(device)
        
        
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
            # loss.backward()
            # optimizer.first_step(zero_grad=True)
            # criterion(model(images), labels).backward()
            # optimizer.second_step(zero_grad=True)
            # record loss
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # global_step += 1
        scheduler.step()
            # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
#         if step % 100 == 0 or step == (len(train_loader)-1):
#             print('Epoch: [{0}][{1}/{2}] '
#                       'Data {data_time.val:.6f} ({data_time.avg:.6f}) '
#                       'Elapsed {remain:s} '
#                       'Loss: {loss.val:.6f}({loss.avg:.6f}) '
#                       'LR: {lr:.6f}  '
#                       .format(
#                        epoch+1, step, len(train_loader), batch_time=batch_time,
#                        data_time=data_time, loss=losses,
#                        remain=timeSince(start, float(step+1)/len(train_loader)),
#                        lr=scheduler.get_lr()[0],
#                        ))
        torch.cuda.empty_cache()
        gc.collect()
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{losses.avg:0.4f}',
                        lr=f'{current_lr:0.8f}',
                        gpu_mem=f'{mem:0.2f} GB')

    return losses.avg

def valid_fn_no_sigmoid(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append((outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels


def valid_fn(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(torch.sigmoid(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels
def valid_fn_two(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(F.softmax(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels
def valid_fn_flip(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        images = torch.flip(images, [3])
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(torch.sigmoid(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels

In [15]:
from exhaustive_weighted_random_sampler import ExhaustiveWeightedRandomSampler
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0
    
def dfs_freeze(module):
    for param in module.parameters():
        param.requires_grad = False
        
def dfs_unfreeze(module):
    for param in module.parameters():
        param.requires_grad = True
    
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

set_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gc.collect()
torch.cuda.empty_cache()
for fold in [0, 1, 2, 3, 4]:
    LOGGER.info(f"Fold: {fold}")
    model = Model(model_name=CFG.model_name).to(device)
    # model = ModelVIT().to(CFG.device)
    train_df = df1[df1['fold']!=fold].reset_index(drop=True)
    valid_df = df[df['fold']==fold].reset_index(drop=True)
    # print(len(valid_df))
    LOGGER.info(f"Len train df: {len(train_df)}")
    cancer_labels = train_df['cancer'].values.tolist()
    class_zero =len(train_df[train_df['cancer']==0])
    class_one = len(train_df[train_df['cancer']==1])
    class_sample_count = np.array([class_zero, class_one*32])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in cancer_labels])
    samples_weight = torch.from_numpy(samples_weight)
    samples_weight = samples_weight.double()
#     print(samples_weight)
    sampler = ExhaustiveWeightedRandomSampler(samples_weight, len(samples_weight))
    
    train_dataset = BreastDataset(train_df, transforms=data_transforms['train'])

    train_loader = DataLoader(train_dataset, batch_size = CFG.train_bs,
                                  num_workers=1, shuffle=True, pin_memory=True, drop_last=True)
    
    valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

    valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                  num_workers=1, shuffle=False, pin_memory=True, drop_last=False)
    
    LEN_DL_TRAIN = len(train_loader)
    best_f1 = 0
    best_metric = 0
    total_epoch = 15
    # checkpoint = torch.load("tf_efficientnetv2_b2_fold_0_model_epoch_3_0.3669_0.309.pth")
    # model.load_state_dict(checkpoint['state_dict'])
    # base_optimizer = AdamP
    # optimizer = SAM(model.parameters(),
    #                 base_optimizer,
    #                 rho=0.05,
    #                 lr=1e-4,
    #                 weight_decay=0.0,
    #                 nesterov=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4, weight_decay = 5e-4)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)  
    # optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = 1*LEN_DL_TRAIN, num_training_steps =total_epoch*LEN_DL_TRAIN)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_epoch)
    # scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_epoch-1)
    # scheduler = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
    # swa_model = AveragedModel(model)
    # swa_scheduler = SWALR(optimizer, swa_lr=1e-4, anneal_epochs=0)
    # scheduler.load_state_dict(checkpoint['scheduler'])
    criterion = nn.CrossEntropyLoss().to(device)
    # criterion1 = nn.BCEWithLogitsLoss().to(device)
    # criterion = nn.BCEWithLogitsLoss().to(CFG.device)
    LOGGER.info(f"Train bs: {CFG.train_bs}")
    # LOGGER.info(f"Model: {model}")
    
    LOGGER.info(f"optimizer: {optimizer}")
    LOGGER.info(f"total_epoch :{total_epoch}")
#     criterion = FocalLoss().to(device)
    for epoch in range(1, total_epoch+1):
        # if epoch >=7:
        #     swa_model.update_parameters(model)
        #     swa_scheduler.step()
        # else:
        # scheduler.step(epoch-1)
        LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
        loss_train = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)
        # state = {'epoch': epoch, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'scheduler':scheduler.state_dict()}
        # path = f'{CFG.model_name}_fold_{fold}_model_epoch_{epoch}.pth'
        # torch.save(state, path)
        loss_valid, valid_preds, valid_labels = valid_fn_two(valid_loader, model, criterion, device)
        # print(valid_preds)
        valid_preds = valid_preds[:, 1]
        valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
        valid_preds = np.array(valid_preds).flatten()
        
        valid_df['raw_pred'] = valid_preds
        # LOGGER.info(f"Valid loss:{loss_valid:.4f}")
        LOGGER.info(f"Train loss:{loss_train:.4f}, Valid loss:{loss_valid:.4f}")
        # print(valid_df.head())
        grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
        grp_df['cancer'] = grp_df['cancer'].astype(np.int)
        valid_labels_mean = grp_df['cancer'].values.tolist()
        valid_preds_mean = grp_df['raw_pred'].values.tolist()
        # print(valid_labels[:5], valid_preds_mean[:5])
        val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
        LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
        best_metric_mean_at_epoch = 0
        best_threshold_mean = 0
        best_auc = 0
        best_cf = None
        for i in np.arange(0.001, 0.599, 0.001):
            valid_argmax = (valid_preds_mean>i).astype(np.int32)
    #             print(valid_argmax)
            val_metric = pfbeta(valid_labels_mean, valid_argmax)
            val_acc = accuracy_score(valid_labels_mean, valid_argmax)
            val_f1 = f1_score(valid_labels_mean, valid_argmax)
            val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
            cf = confusion_matrix(valid_labels_mean, valid_argmax)
            if val_metric> best_metric_mean_at_epoch:
                best_metric_mean_at_epoch = val_metric
                best_threshold_mean = i
                best_auc = val_auc
                best_cf = cf
            # print(f"Threshold: {i:.4f}, val_acc: {val_acc:.4f}, val_f1: {val_f1:.4f}, val_auc: {val_auc:.4f}, val_metric: {val_metric:.4f}")
        LOGGER.info(f"Best metric at epoch {epoch}: {best_metric_mean_at_epoch:.4f} {best_threshold_mean:.4f} {best_auc:.4f}")
        LOGGER.info(f"Cf: {best_cf}")
    #         print(f"Train loss: {loss_train:.4f}, eval loss: {loss_valid.avg:.4f}") 
    #         print(f"Accuracy score: {val_acc:.4f}, f1 score: {val_f1:.4f}")
    #         print(f"Comp metric: {val_metric:.4f}")
        if best_metric_mean_at_epoch > best_metric:

            LOGGER.info(f"Model improve: {best_metric:.4f} -> {best_metric_mean_at_epoch:.4f}")
            best_metric = best_metric_mean_at_epoch
        state = {'epoch': epoch, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'scheduler':scheduler.state_dict()}
        path = f'{CFG.model_name}_fold_{fold}_model_epoch_{epoch}_{best_metric_mean_at_epoch:.4f}_{best_threshold_mean:.3f}.pth'
        torch.save(state, path)
    #     loss_valid, valid_preds, valid_labels = valid_fn_no_sigmoid(valid_loader, model, criterion, device)
        
    #     # valid_preds = valid_preds[:, 1]
    #     valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    #     valid_preds = np.array(valid_preds).flatten()
        
    #     valid_df['raw_pred'] = valid_preds
    #     # LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    #     LOGGER.info(f"Train loss:{loss_train:.4f}, Valid loss:{loss_valid:.4f}")
    #     # print(valid_df.head())
    #     grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    #     grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    #     valid_labels_mean = grp_df['cancer'].values.tolist()
    #     valid_preds_mean = grp_df['raw_pred'].values.tolist()
    #     valid_preds_mean = [sigmoid(x) for x in valid_preds_mean]
    #     # print(valid_labels[:5], valid_preds_mean[:5])
    #     val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    #     LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    #     best_metric_mean_at_epoch = 0
    #     best_threshold_mean = 0
    #     best_auc = 0
    #     best_cf = None
    #     for i in np.arange(0.001, 0.999, 0.001):
    #         valid_argmax = (valid_preds_mean>i).astype(np.int32)
    # #             print(valid_argmax)
    #         val_metric = pfbeta(valid_labels_mean, valid_argmax)
    #         val_acc = accuracy_score(valid_labels_mean, valid_argmax)
    #         val_f1 = f1_score(valid_labels_mean, valid_argmax)
    #         val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
    #         cf = confusion_matrix(valid_labels_mean, valid_argmax)
    #         if val_metric> best_metric_mean_at_epoch:
    #             best_metric_mean_at_epoch = val_metric
    #             best_threshold_mean = i
    #             best_auc = val_auc
    #             best_cf = cf
    #         # print(f"Threshold: {i:.4f}, val_acc: {val_acc:.4f}, val_f1: {val_f1:.4f}, val_auc: {val_auc:.4f}, val_metric: {val_metric:.4f}")
    #     LOGGER.info(f"Best metric at epoch {epoch}: {best_metric_mean_at_epoch:.4f} {best_threshold_mean:.4f} {best_auc:.4f}")
    #     LOGGER.info(f"Cf: {best_cf}")
    # #         print(f"Train loss: {loss_train:.4f}, eval loss: {loss_valid.avg:.4f}") 
    # #         print(f"Accuracy score: {val_acc:.4f}, f1 score: {val_f1:.4f}")
    # #         print(f"Comp metric: {val_metric:.4f}")
    #     if best_metric_mean_at_epoch > best_metric:

    #         LOGGER.info(f"Model improve: {best_metric:.4f} -> {best_metric_mean_at_epoch:.4f}")
    #         best_metric = best_metric_mean_at_epoch
    #     state = {'epoch': epoch+1, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'scheduler':scheduler.state_dict()}
    #     path = f'{CFG.model_name}_fold_{fold}_model_epoch_{epoch+1}_{best_metric_mean_at_epoch:.4f}_{best_threshold_mean:.3f}.pth'
    #     torch.save(state, path)
#     torch.optim.swa_utils.update_bn(train_loader, swa_model, device=device)
        

Fold: 0


> SEEDING DONE


Len train df: 44652
Train bs: 16
optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.0001
    lr: 0.0
    maximize: False
    weight_decay: 0.0005
)
total_epoch :15
Epoch: 1/15
Val: 100%|██████████| 172/172 [03:34<00:00,  1.25s/it, eval_loss=0.1136, gpu_mem=11.47 GB]
Train loss:0.2181, Valid loss:0.1136
Val metric mean prob: 0.0449
Best metric at epoch 1: 0.0964 0.1180 0.5771
Cf: [[4312  354]
 [  77   23]]
Model improve: 0.0000 -> 0.0964
Epoch: 2/15
Val: 100%|██████████| 172/172 [03:29<00:00,  1.22s/it, eval_loss=0.0872, gpu_mem=11.47 GB]
Train loss:0.1571, Valid loss:0.0872
Val metric mean prob: 0.1265
Best metric at epoch 2: 0.3005 0.1650 0.6381
Cf: [[4602   64]
 [  71   29]]
Model improve: 0.0964 -> 0.3005
Epoch: 3/15
Val: 100%|██████████| 172/172 [03:35<00:00,  1.25s/it, eval_loss=0.1040, gpu_mem=11.47 GB]
Train loss:0.1381, Valid loss:0.1040
Val metric mean prob: 0.1810
Best metric